## ResNets

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample = None):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

        self.batchNorm1 = nn.BatchNorm2d(num_features=out_channels)

        self.relu = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False)

        self.batchNorm2 = nn.BatchNorm2d(num_features=out_channels)

        self.downsample = downsample

    def forward(self, x):
        identity = x

        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.batchNorm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batchNorm2(out)
        out = out + identity
        out = self.relu(out)
        
        return out

In [13]:
class CustomResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomResNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.relu = nn.ReLU()
        self.maxPool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(in_channels=64, out_channels=64, blocks=2)
        self.layer2 = self._make_layer(in_channels=64, out_channels=128, blocks=2, stride=2)
        self.layer3 = self._make_layer(in_channels=128, out_channels=256, blocks=2, stride=2)
        self.layer4 = self._make_layer(in_channels=256, out_channels=512, blocks=2, stride=2)

        self.avgPool = nn.AdaptiveAvgPool2d((1,1))

        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        downSample = None
        if stride != 1 or in_channels != out_channels:
            downSample = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_features=out_channels)
            )
        
        layers = [ResidualBlock(in_channels=in_channels, out_channels=out_channels, stride=stride, downsample=downSample)]

        for _ in range(1, blocks):
            layers.append(ResidualBlock(in_channels=out_channels, out_channels=out_channels))

        return nn.Sequential(*layers)
    

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxPool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgPool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [14]:
model = CustomResNet()

In [15]:
use_custom_model = True
if use_custom_model:
    model = CustomResNet().to(device=device)
else:
    model = models.resnet18(pretrained = True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features=num_ftrs, out_features=256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=10)
    )

    model = model.to(device=device)

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=1e-3)

In [18]:
epochs = 2
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device=device), labels.to(device=device)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch: {epoch+1}, Loss: {running_loss/len(train_loader)}")

 50%|█████     | 1/2 [00:19<00:19, 19.47s/it]

Epoch: 1, Loss: 0.9855749206164913


100%|██████████| 2/2 [00:38<00:00, 19.49s/it]

Epoch: 2, Loss: 0.8087683447334163


In [19]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device=device), labels.to(device=device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test accuracy: {100 * correct / total}")

Test accuracy: 71.63
